In [3]:
import genjax
from genjax import dippl
import jax
import adevjax

console = genjax.pretty()
key = jax.random.PRNGKey(314159)

In [5]:
@genjax.gen
def model(p):
    v = dippl.flip_enum(p) @ "v"
    return v


lifted = dippl.chm_dist(model, genjax.select("v"), None)


def emitter(key, p):
    key, sub_key = jax.random.split(key)

    @adevjax.adev
    def flip_loss(p):
        (w, v) = lifted.random_weighted(sub_key, p)
        b = v.get_leaf_value()["v"]
        return jax.lax.cond(
            b,
            lambda _: w * 0.0,
            lambda p: w * (-p / 2.0),
            p,
        )

    return adevjax.E(flip_loss).grad_estimate(key, p)


jax.jit(emitter)(key, (0.3,))

(Array(0.221335, dtype=float32),)